In [0]:
# import findspark
# findspark.init('/spark-3.5.1-bin-hadoop3')
from pyspark import *
from pyspark.sql import SparkSession, Row
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, DateType, TimestampType, LongType
from pyspark.sql.types import ArrayType, DoubleType, BooleanType, DecimalType
from pyspark.sql.functions import regexp_extract, split, from_unixtime, col, avg, min, max, desc
from pyspark.sql.functions import grouping, explode, array_contains
from pyspark.sql.functions import mean, stddev, skewness, kurtosis
import matplotlib.pyplot as plt
import pandas as pd
import random


# spark = SparkSession.builder.appName("analytics").getOrCreate()

# Arboles de Decisión

## Entrenamiento

### Generación de Datos

Primero generamos un conjunto de datos aleatorios que usaria Spark para entrenar el arbol de decisión

Características del conjunto de datos:

* Edad (entero): La edad del cliente.
* Saldo (entero): El saldo de la cuenta del cliente en el banco.
* Empleo (categórico): Tipo de empleo (ej. "admin", "técnico", "servicios").
* Educación (categórico): Nivel educativo (ej. "primaria", "secundaria", "universitaria").
* Suscripción (binario): Si el cliente se suscribió a un depósito a plazo (1) o no (0).

In [0]:
# Generar datos
empleos = ["admin", "tecnico", "servicios", "gerente"]
educacion = ["primaria", "secundaria", "universitaria"]

data = [
    Row(edad=random.randint(18, 65),
        saldo=random.randint(-500, 5000),
        empleo=random.choice(empleos),
        educacion=random.choice(educacion),
        suscripcion=random.choice([0, 1]))
    for _ in range(300)
]

df = spark.createDataFrame(data)

# Guardar los datos en CSV
df.write.csv("/Volumes/big_data_ii_2025/spark_examples/spark_data/datos_banco.csv", header=True, mode="overwrite")

In [0]:
df.printSchema()

In [0]:
df.display(5)

## Preparar los datos

 Utilizamos StringIndexer para convertir las categorías en índices numéricos basados en la frecuencia de cada categoría, asignando los índices más bajos a las categorías más frecuentes.

Posteriormente, OneHotEncoder transforma estos índices numéricos en vectores binarios para evitar que el modelo interprete las variables categóricas como si tuvieran un orden o jerarquía. Cada categoría se representa con un vector donde sólo un elemento es '1' y el resto son '0', 

In [0]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler

# Indexar las columnas categóricas
# indexer = StringIndexer(inputCols=["empleo", "educacion"], outputCols=["empleo_idx", "educacion_idx"])
# indexer_empleo = StringIndexer().setInputCol("empleo").setOutputCol("empleo_idx")
# indexer_educacion = StringIndexer().setInputCol("educacion").setOutputCol("educacion_idx")
indexer_empleo = StringIndexer(inputCol="empleo", outputCol="empleo_idx")
indexer_educacion = StringIndexer(inputCol="educacion", outputCol="educacion_idx")
# df_indexed = indexer.fit(df).transform(df)
df_indexed = indexer_empleo.fit(df).transform(df)
df_indexed = indexer_educacion.fit(df_indexed).transform(df_indexed)

# Aplicar OneHotEncoder a las columnas indexadas
encoder = OneHotEncoder(inputCols=["empleo_idx", "educacion_idx"], outputCols=["empleo_enc", "educacion_enc"])
df_encoded = encoder.fit(df_indexed).transform(df_indexed)

# Assembler para combinar todas las características en una sola columna de vectores
assembler = VectorAssembler(
    inputCols=["edad", "saldo", "empleo_enc", "educacion_enc"],
    outputCol="features"
)
df_final = assembler.transform(df_encoded)

# Seleccionar las columnas necesarias para el modelo
df_model = df_final.select("features", "suscripcion")

# df_model.show(5)
display(df_model.limit(5))

In [0]:
df_final.show(5)

### Entrenar el modelo de árbol de decisión




In [0]:
from pyspark.ml.classification import DecisionTreeClassifier

# Crear y entrenar el modelo de árbol de decisión
dt = DecisionTreeClassifier(labelCol="suscripcion", featuresCol="features")
model = dt.fit(df_model)

# Mostrar el árbol de decisión
print("Árbol de decisión modelado:")
print(model.toDebugString)


## Clasificación

### Crear un ejemplo

In [0]:
# Crear un DataFrame con un ejemplo
ejemplo = spark.createDataFrame([
    (30, 1500, "admin", "universitaria"),(30, 1500, "servicios", "secundaria")
], ["edad", "saldo", "empleo", "educacion"])

# Mostrar el nuevo ejemplo
ejemplo.show()


### Preprocesar el ejemplo

In [0]:
# Aplicar StringIndexer
ejemplo_indexed = indexer.fit(ejemplo).transform(ejemplo)

# Aplicar OneHotEncoder
ejemplo_encoded = encoder.fit(ejemplo_indexed).transform(ejemplo_indexed)

# Aplicar VectorAssembler
ejemplo_final = assembler.transform(ejemplo_encoded)

# Seleccionar la columna 'features'
ejemplo_final = ejemplo_final.select("features")
ejemplo_final.show()


### Clasificar el ejemplo

In [0]:
# Hacer predicciones
predicciones = model.transform(ejemplo_final)

# Mostrar las predicciones
predicciones.select("prediction").show()
